# 池化层pooling

当处理图像时，通常希望**逐渐降低**隐藏表示的空间分辨率、聚集信息，这样随着在神经网络中层数的上升，每个神经元对其敏感的**感受野就越大**  

机器学习任务通常会跟全局图像的问题有关，故最后一层的神经元应该对整个输入的全局敏感。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层

- 卷积对位置敏感
    - 前例：检测垂直边缘——1像素移位，就会导致0输出
- 需要一定程度的**平移不变性**
    - 照明、物体位置、比例、外观等因图像而异

### 二维最大池
- 返回滑动窗口中的最大值
    - 返回**滑动窗口中的最大值**
- 垂直边缘检测——卷积——$2\times2$最大池化
    - 可容1像素移动

### 填充，步幅和多个通道
- 池化层和卷积层类似，**都具有填充和步幅**
- 没有可学习的参数（没有kernel）
- 在每个输入通道应用池化层以获得相应的输出通道
- **输出通道数 = 输入通道数**（不会融合通道，融合交给卷积）

### 平均池化层
- 最大池化层；每个窗口中最强的模式信号
- 平均池化层：将最大池化层中‘最大’操作替换为**‘平均’**

### 总结
- 池化层返回窗口中最大或平均值
- **缓解卷积层对位置的敏感性**，一般在卷积层之后
- 同样有**窗口大小**、**填充**和**步幅**作为超参数

## 代码

**实现池化层的正向传播**

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i+p_h, j:j+p_w].mean()
    return Y

**验证二维最大池化层和平均池化层的输出**

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [3]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

**填充和步幅**——使用深度学习框架中**内置**的二维最大池化层

In [4]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

**默认情况下**，深度学习框架的**池化层**，如果不指定`stride`，**步幅会默认等于池化窗口大小**

In [5]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

C:\Users\lenovo\.conda\envs\fxr_env2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[[[10.]]]])

**填充和步幅可以手动设定**

In [6]:
pool2d = nn.MaxPool2d(3, padding=1, stride=(2, 3))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

**设定一个任意大小的矩形池化窗口，并且分别设定填充和步幅的高度和宽度**

In [7]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

#### 多个通道
**池化层在每个输入通道上单独运算**，而不是像卷积层一样在通道上对输入进行汇总——输入通道=输出通道
- 在**通道维度**上连结张量`X`和`X + 1`，以构建具有2个通道的输入

In [8]:
X = torch.cat([X, X +1], 1)
# X = torch.cat((X, X +1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [9]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

### 注意
- 池化层目前用的越来越少——首先会对数据本身做很多变化（扰动操作）+卷积层的步幅就可以减少计算量
- 池化时窗口有没有重叠都无所谓了

### 练习
1. 尝试将平均池化层作为卷积层的特殊情况实现

In [ ]:
import torch

from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.pool = nn.Conv2d(6, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.lin1 = nn.Linear(16 * 5 * 5, 128)
        self.lin2 = nn.Linear(128, 64)
        self.lin3 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(X, (2, 2)) # 平均池化层
        x = F.relu(self.conv2(x))
        